# Import Libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer

In [ ]:
consumer_data = pd.read_csv('/kaggle/input/financial-consumer-csv/Financial Consumer Complaints.csv')
consumer_data

# Wrangling Data

In [ ]:
def wrangle_consumer_data(df):
    # Drop unnecessary columns
    df = df.drop(['Tags', 'Consumer consent provided?', 'ZIP code'], axis=1)
    
    # Replace spaces with underscores in column names
    df.columns = [col.replace(' ', '_') for col in df.columns]
    
    # Convert date columns to datetime
    df['Date_Sumbited'] = pd.to_datetime(df['Date_Sumbited'], format='%m/%d/%y')
    df['Date_Received'] = pd.to_datetime(df['Date_Received'], format='%m/%d/%y')
    
    # Replace empty string values in 'Sub-product' and 'Sub-issue'
    df['Sub-product'] = df['Sub-product'].replace('""', "I do not know")
    df['Sub-issue'] = df['Sub-issue'].replace('""', "I do not know")
    
    # Fill missing values in 'Company_public_response' with a default text
    df['Company_public_response'].fillna('Pending to provide a public response', inplace=True)
    
    # Find the most frequent value in the 'State' column
    most_frequent_state = df['State'].mode()[0]

    # Fill missing values with the most frequent state
    df['State'].fillna(value=most_frequent_state, inplace=True)
    
    # Drop rows where 'Sub-issue' is NaN
    df.dropna(subset=['Sub-issue'], inplace=True)
    
    # Replace the category values with Numerical to impute missing Values
    dict_ = {
    'Yes':1,
    'No': 0
    }
    df['Consumer_disputed?'] = df['Consumer_disputed?'].map(dict_)

    # Select the column to impute
    column_to_impute = 'Consumer_disputed?'
    # Create a copy of the dataset with the selected column
    impute_data = df[[column_to_impute]].copy()

    # Create an instance of the KNNImputer with the desired parameters
    imputer = KNNImputer(n_neighbors=5)

    # Fit and transform the selected column using the imputer
    impute_data[column_to_impute] = imputer.fit_transform(impute_data[[column_to_impute]])

    # Replace the original column in the dataset with the imputed values
    df[column_to_impute] = impute_data[column_to_impute]

    # Creating a dictionary for replacement
    replacement_dict = {value: 'yes' if value >= 0.5 else 'no' for value in df['Consumer_disputed?'].unique()}

    # Applying the replacement to the 'Consumer_disputed?' column
    df['Consumer_disputed?'] = df['Consumer_disputed?'].map(replacement_dict)
    
    df.drop_duplicates(inplace=True)

    return df

In [ ]:
consumer_data = wrangle_consumer_data(consumer_data)

In [ ]:
consumer_data.head()

In [ ]:
consumer_data.info()

In [ ]:
# Save the Datast to use in Power BI Dashboard
consumer_data.to_csv('Financial_Consumer_data.csv')